In [11]:
#distribution 
import os
from ROOT import *
import numpy as np
import uncertainties as unc # propagate uncertainties
from uncertainties import unumpy as unp # array operations for type ufloat

doreweight = 3    #decide if we want to do the reweighting process

# var = "ntrk"  #change the var name according to the var you want to read
mc_type = "pythia"   #by setting it as "SF" or "MC", it will automatically making scale factor plots or MC closure plots
# var = "c1"  #by setting it as bdt (or ntrk,width,c1..), it will read the corresponding histogram, but remember to change the TLine range according to X-axis of different variable, one can check it by browsing the histograms in root file.
var="eta"



# 0 mean nominal result(no eta reweight)
# 1 mean flatten reweight
# 2 mean customize reweight

eta_reweight =0

    
if eta_reweight == 0:
    nevents_eta = []
    for i in range(6):
        nevents_eta.append(np.ones(3))
    

if eta_reweight == 1:
    nevents_eta = np.load("mc_eta_events.npy")
    
if eta_reweight == 2:
    nevents_eta = []
    for i in range(6):
        nevents_eta.append([1,2,3]) 
    
if eta_reweight == 3:
    nevents_eta = []
    for i in range(6):
        nevents_eta.append([1,10,20]) 
        
        
eta_bin = [0,0.4,0.8,1.4]        
#ntrackall = TFile("../root/dijet_powpyt_mono-a.root")
fdijetMC = TFile("/eos/user/r/rqian/dijet-mono-result/pythia-eta-test/dijet_pythia_etatest_1-4.root")

fdijetData = TFile("/eos/user/r/rqian/dijet-mono-result/data-eta-test/dijet_data_1-4.root")

#ntrackall3 = TFile("../root/dijet_data_17Feb28.root")
#ntrackall3 = TFile("../root/dijet_data_18_prescale.root")



def myText(x,y,text,color =1):
    l = TLatex()
    l.SetTextSize(0.025)
    l.SetNDC()
    l.SetTextColor(color)
    l.DrawLatex(x,y,text)
    pass


#convert histogram and error into unp.uarray
#if sample is pyroot input, GetBinError returns correct result
def unc_array(hist):
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = hist.GetBinError(j)
    result = unp.uarray(value,error)
    return(result)

#convert histogram and error into unp.uarray
#if sample is uproot input, err is the sumw2 of the corresponding histogram
def unc_array_err(hist,err):
    value = np.zeros(hist.GetNbinsX())
    error = np.zeros(hist.GetNbinsX())
    for j in range(1,hist.GetNbinsX()+1):
        value[j-1] = hist.GetBinContent(j)
        error[j-1] = np.sqrt(err.GetBinContent(j))
    result = unp.uarray(value,error)
    return(result)

def set_hist_error(hist,unc):
    for i in range(1,hist.GetNbinsX()+1):
        hist.SetBinError(i,unp.std_devs(unc[i-1]))
        
path = mc_type
isExists=os.path.exists(mc_type)
if not isExists:
        os.makedirs(mc_type) 
        
bins = np.array([500.,600.,800.,1000.,1200.,1500.,2000.])
bin = np.array([500,600,800,1000,1200,1500,2000])


def ReadingDijetDataEta(file0,pt,var,name,index):
    histo_dic = {}
    higher_data = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var)
    lower_data = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var)

    higher_data.SetName(pt+"_LeadingJet_Forward_Data_"+str(index)+var+name)
    lower_data.SetName(pt+"_LeadingJet_Central_Data_"+str(index)+var+name)
    
    higher_data_2 = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var)
    lower_data_2 = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var)
    
    higher_data_2.SetName(pt+"_SubJet_Forward_Data_"+str(index)+var+name)
    lower_data_2.SetName(pt+"_SubJet_Central_Data_"+str(index)+var+name)
    
    higher_data.Add(higher_data_2)
    lower_data.Add(lower_data_2)
    

    higher_data_err = file0.Get(pt+"_LeadingJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_err = file0.Get(pt+"_LeadingJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_err.SetName(pt+"_LeadingJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_err.SetName(pt+"_LeadingJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_2_err = file0.Get(pt+"_SubJet_Forward_Data_"+str(index)+"_"+var+"_err")
    lower_data_2_err = file0.Get(pt+"_SubJet_Central_Data_"+str(index)+"_"+var+"_err")

    higher_data_2_err.SetName(pt+"_SubJet_Forward_Data_"+var+"_err"+str(index)+var+name)
    lower_data_2_err.SetName(pt+"_SubJet_Central_Data_"+var+"_err"+str(index)+var+name)

    higher_data_err.Add(higher_data_2_err)
    lower_data_err.Add(lower_data_2_err)
    
    for i in range(1,higher_data.GetNbinsX()+1):
        higher_data.SetBinError(i,np.sqrt(higher_data_err.GetBinContent(i)))
        lower_data.SetBinError(i,np.sqrt(lower_data_err.GetBinContent(i)))
        
    histo_dic["hd"] =  higher_data
    histo_dic["ld"] =  lower_data

    return(histo_dic)        


def ReadingDijetMCEta(file0,pt,var,name,index):
    histo_dic = {}
    higher_quark = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var)


    higher_quark.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+var+name)
    lower_quark.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+var+name)
    higher_gluon.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+var+name)
    

    higher_quark_2 = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var)
    lower_quark_2 = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var)
    higher_gluon_2 = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var)
    lower_gluon_2 = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var)
    higher_b_quark_2 = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var)
    lower_b_quark_2 = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var)
    higher_c_quark_2 = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var)
    lower_c_quark_2 = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var)

    higher_quark_2.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+var+name)
    lower_quark_2.SetName(pt+"_SubJet_Central_Quark_"+str(index)+var+name)
    higher_gluon_2.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+var+name)
    lower_gluon_2.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+var+name)
    higher_b_quark_2.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+var+name)
    lower_b_quark_2.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+var+name)
    higher_c_quark_2.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+var+name)
    lower_c_quark_2.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+var+name)

    higher_quark.Add(higher_quark_2)
    higher_gluon.Add(higher_gluon_2)
    higher_b_quark.Add(higher_b_quark_2)
    higher_c_quark.Add(higher_c_quark_2)

    lower_quark.Add(lower_quark_2)
    lower_gluon.Add(lower_gluon_2)
    lower_b_quark.Add(lower_b_quark_2)
    lower_c_quark.Add(lower_c_quark_2)
 
            
    higher_b_quark.Add(higher_c_quark)
    lower_b_quark.Add(lower_c_quark)

    higher_quark_err = file0.Get(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_err = file0.Get(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_err = file0.Get(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_err = file0.Get(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_err = file0.Get(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_err = file0.Get(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_err = file0.Get(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_err = file0.Get(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err")



    
    higher_quark_err.SetName(pt+"_LeadingJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_err.SetName(pt+"_LeadingJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_err.SetName(pt+"_LeadingJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_err.SetName(pt+"_LeadingJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_err.SetName(pt+"_LeadingJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_err.SetName(pt+"_LeadingJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_err.SetName(pt+"_LeadingJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_err.SetName(pt+"_LeadingJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_2_err = file0.Get(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err")
    lower_quark_2_err = file0.Get(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err")
    higher_gluon_2_err = file0.Get(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err")
    lower_gluon_2_err = file0.Get(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err")
    higher_b_quark_2_err = file0.Get(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err")
    lower_b_quark_2_err = file0.Get(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err")
    higher_c_quark_2_err = file0.Get(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err")
    lower_c_quark_2_err = file0.Get(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err")
    
    higher_quark_2_err.SetName(pt+"_SubJet_Forward_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_quark_2_err.SetName(pt+"_SubJet_Central_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_gluon_2_err.SetName(pt+"_SubJet_Forward_Gluon_"+str(index)+"_"+var+"_err"+name)
    lower_gluon_2_err.SetName(pt+"_SubJet_Central_Gluon_"+str(index)+"_"+var+"_err"+name)
    higher_b_quark_2_err.SetName(pt+"_SubJet_Forward_B_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_b_quark_2_err.SetName(pt+"_SubJet_Central_B_Quark_"+str(index)+"_"+var+"_err"+name)
    higher_c_quark_2_err.SetName(pt+"_SubJet_Forward_C_Quark_"+str(index)+"_"+var+"_err"+name)
    lower_c_quark_2_err.SetName(pt+"_SubJet_Central_C_Quark_"+str(index)+"_"+var+"_err"+name)

    higher_quark_err.Add(higher_quark_2_err)
    higher_gluon_err.Add(higher_gluon_2_err)
    higher_b_quark_err.Add(higher_b_quark_2_err)
    higher_c_quark_err.Add(higher_c_quark_2_err)

    lower_quark_err.Add(lower_quark_2_err)
    lower_gluon_err.Add(lower_gluon_2_err)
    lower_b_quark_err.Add(lower_b_quark_2_err)
    lower_c_quark_err.Add(lower_c_quark_2_err)
 
            
    higher_b_quark_err.Add(higher_c_quark_err)
    lower_b_quark_err.Add(lower_c_quark_err)
    
    for i in range(1,higher_quark.GetNbinsX()+1):
        higher_quark.SetBinError(i,np.sqrt(higher_quark_err.GetBinContent(i)))
        higher_gluon.SetBinError(i,np.sqrt(higher_gluon_err.GetBinContent(i)))
        higher_b_quark.SetBinError(i,np.sqrt(higher_b_quark_err.GetBinContent(i)))
        lower_quark.SetBinError(i,np.sqrt(lower_quark_err.GetBinContent(i)))
        lower_gluon.SetBinError(i,np.sqrt(lower_gluon_err.GetBinContent(i)))
        lower_b_quark.SetBinError(i,np.sqrt(lower_b_quark_err.GetBinContent(i)))

    
    histo_dic["hq"] =  higher_quark
    histo_dic["hg"] =  higher_gluon
    histo_dic["ho"] =  higher_b_quark
    
    histo_dic["hmc"] =  higher_quark.Clone()
    histo_dic["hmc"].SetName(pt+"higher_all_jets"+var+"_"+name)
    histo_dic["hmc"].Add(higher_gluon)
    histo_dic["hmc"].Add(higher_b_quark)
    
    histo_dic["lq"] =  lower_quark
    histo_dic["lg"] =  lower_gluon
    histo_dic["lo"] =  lower_b_quark
    
    histo_dic["lmc"] =  lower_quark.Clone()
    histo_dic["lmc"].SetName(pt+"lower_all_jets"+var+"_"+name)
    histo_dic["lmc"].Add(lower_gluon)
    histo_dic["lmc"].Add(lower_b_quark)
    
    return(histo_dic)


for k in range(0,6):   #for only dijet event, start from jet pT>500 GeV
        #for i in range(13):	#for gamma+jet combined with dijet event, start from jet pT>0 GeV
                print(min)
                min = bin[k]
                max = bin[k+1]
                nominal_jets =  ReadingDijetMCEta(fdijetMC,str(min),var,"0","0")
                
                data_jets = ReadingDijetDataEta(fdijetData,str(min),var,"0","0")
    
                
                higher_quark = nominal_jets["hq"]
                lower_quark = nominal_jets["lq"]
                higher_gluon = nominal_jets["hg"]
                lower_gluon = nominal_jets["lg"]
                higher_other = nominal_jets["ho"]
                lower_other = nominal_jets["lo"]        
                higher_mc = nominal_jets["hmc"]
                lower_mc = nominal_jets["lmc"]        
    
                
                higher_data_TH1I = data_jets["hd"]
                higher_data = higher_quark.Clone(higher_data_TH1I.GetName()+"TH1F")
                for j in range(1,higher_data.GetNbinsX()+1):
                        higher_data.SetBinContent(j,float(higher_data_TH1I.GetBinContent(j)))
                        higher_data.SetBinError(j,float(higher_data_TH1I.GetBinError(j)))
                        
                 
                lower_data_TH1I = data_jets["ld"]
                lower_data = lower_quark.Clone(lower_data_TH1I.GetName()+"TH1F")
                for j in range(1,higher_data.GetNbinsX()+1):
                        lower_data.SetBinContent(j,float(lower_data_TH1I.GetBinContent(j)))
                        lower_data.SetBinError(j,float(lower_data_TH1I.GetBinError(j))) 
            
                higher_quark.Scale(nevents_eta[k][0])
                higher_gluon.Scale(nevents_eta[k][0])
                higher_other.Scale(nevents_eta[k][0])
                higher_mc.Scale(nevents_eta[k][0])
                lower_quark.Scale(nevents_eta[k][0])
                lower_gluon.Scale(nevents_eta[k][0])
                lower_other.Scale(nevents_eta[k][0])
                lower_mc.Scale(nevents_eta[k][0])
                higher_data.Scale(nevents_eta[k][0])
                lower_data.Scale(nevents_eta[k][0])
                
                total_mc = higher_mc.Clone()
                total_mc.Add(lower_mc)

                total_data = higher_data.Clone()
                total_data.Add(lower_data)
                
                for e in range(1,len(eta_bin)-1):
        
                    print(var,str(eta_bin[e]))
                    
                    nominal_jets_add =  ReadingDijetMCEta(fdijetMC,str(min),var,str(eta_bin[e]),str(eta_bin[e]))
                    

                    data_jets_add = ReadingDijetDataEta(fdijetData,str(min),var,str(eta_bin[e]),str(eta_bin[e]))
                    
                    higher_data_TH1I_add = data_jets_add["hd"]
                    higher_data_add = higher_quark.Clone(higher_data_TH1I_add.GetName()+"TH1F")
                    for j in range(1,higher_data.GetNbinsX()+1):
                            higher_data_add.SetBinContent(j,float(higher_data_TH1I_add.GetBinContent(j)))
                            higher_data_add.SetBinError(j,float(higher_data_TH1I_add.GetBinError(j)))
                            
                            
                    lower_data_TH1I_add = data_jets_add["ld"]
                    lower_data_add = lower_quark.Clone(lower_data_TH1I_add.GetName()+"TH1F")
                    for j in range(1,higher_data.GetNbinsX()+1):
                            lower_data_add.SetBinContent(j,float(lower_data_TH1I_add.GetBinContent(j)))
                            lower_data_add.SetBinError(j,float(lower_data_TH1I_add.GetBinError(j)))   
                    
                    # flatten the eta distribution
                    nominal_jets_add["hq"].Scale(nevents_eta[k][e])
                    nominal_jets_add["hg"].Scale(nevents_eta[k][e])
                    nominal_jets_add["ho"].Scale(nevents_eta[k][e])
                    nominal_jets_add["hmc"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lq"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lg"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lo"].Scale(nevents_eta[k][e])
                    nominal_jets_add["lmc"].Scale(nevents_eta[k][e])
                    higher_data_add.Scale(nevents_eta[k][e])
                    lower_data_add.Scale(nevents_eta[k][e])
                    
                    higher_quark.Add(nominal_jets_add["hq"])
                    lower_quark.Add(nominal_jets_add["lq"])
                    higher_gluon.Add(nominal_jets_add["hg"])
                    lower_gluon.Add(nominal_jets_add["lg"])
                    higher_other.Add(nominal_jets_add["ho"])
                    lower_other.Add(nominal_jets_add["lo"])        
                    higher_mc.Add(nominal_jets_add["hmc"])
                    lower_mc.Add(nominal_jets_add["lmc"])    
                                        
                        
                    total_mc.Add(nominal_jets_add["hmc"])
                    total_mc.Add(nominal_jets_add["lmc"])
    
                    total_data.Add(higher_data_add)
                    total_data.Add(lower_data_add)
            
                if(total_data.Integral() != 0):
                    total_data.Scale(1./total_data.Integral())
                if(total_mc.Integral() != 0):
                    total_mc.Scale(1./total_mc.Integral())   
                    
                #uncertainty propagation
                higher_quark_unc = unc_array(higher_quark)
                higher_gluon_unc = unc_array(higher_gluon)
                lower_quark_unc = unc_array(lower_quark)
                lower_gluon_unc = unc_array(lower_gluon)
                higher_mc_unc = unc_array(higher_mc)
                lower_mc_unc = unc_array(lower_mc)
            
                higher_data_unc = unc_array(higher_data)
                lower_data_unc = unc_array(lower_data)

                ToT_Fq2_unc = higher_quark_unc.sum()
                ToT_Fg2_unc = higher_gluon_unc.sum()

                ToT_Cq2_unc = lower_quark_unc.sum()
                ToT_Cg2_unc = lower_gluon_unc.sum()

                # calculate the fraction of forward(higher) / central(lower) quark or gluon jet
                fg_unc=ToT_Fg2_unc/(ToT_Fg2_unc+ToT_Fq2_unc)
                cg_unc=ToT_Cg2_unc/(ToT_Cq2_unc+ToT_Cg2_unc)
                fq_unc=1.-fg_unc
                cq_unc=1.-cg_unc

                factor_quark_unc = lower_quark_unc
                factor_gluon_unc = lower_gluon_unc


                #First normalize it 
                higher_quark_unc = higher_quark_unc/higher_quark_unc.sum()
                higher_gluon_unc = higher_gluon_unc/higher_gluon_unc.sum()
                lower_quark_unc = lower_quark_unc/lower_quark_unc.sum()
                lower_gluon_unc = lower_gluon_unc/lower_gluon_unc.sum()
                higher_data_unc = higher_data_unc/higher_data_unc.sum()
                lower_data_unc = lower_data_unc/lower_data_unc.sum()

                higher_quark_hist = higher_quark.Clone()
                higher_gluon_hist = higher_quark.Clone() 
                lower_quark_hist = higher_quark.Clone()
                lower_gluon_hist = higher_quark.Clone()
                higher_data_hist = higher_quark.Clone()
                lower_data_hist = higher_quark.Clone()   


#                 if (doreweight=="Quark"):
                for i in range(1,higher_quark.GetNbinsX()+1):
                        if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
                                factor_gluon_unc[i-1] = higher_gluon_unc[i-1]/lower_gluon_unc[i-1]
                                factor_quark_unc[i-1] = higher_quark_unc[i-1]/lower_quark_unc[i-1]
                        else:
                                factor_gluon_unc[i-1] = unc.ufloat(1, 0)
                                factor_quark_unc[i-1] = unc.ufloat(1, 0)
#                     lower_data_unc=lower_data_unc*factor_quark_unc_pt_bins[k-7]

#                 if (doreweight=="Gluon"):

#                     for i in range(1,higher_quark.GetNbinsX()+1):
#                             if (lower_quark.GetBinContent(i) > 0 and lower_gluon.GetBinContent(i) > 0):
#                                     #print i,higher_quark.GetBinContent(i)/lower_quark.GetBinContent(i),higher_gluon.GetBinContent(i)/lower_gluon.GetBinContent(i)
#                                     factor_gluon_unc[i-1] = higher_gluon_unc[i-1]/lower_gluon_unc[i-1]
#                                     factor_quark_unc[i-1] = higher_quark_unc[i-1]/lower_quark_unc[i-1]
#                             else:
#                                     factor_gluon_unc[i-1] = unc.ufloat(1, 0)
#                                     factor_quark_unc[i-1] = unc.ufloat(1, 0)
#                     lower_data_unc=lower_data_unc*factor_gluon_unc_pt_bins[k-7]

                for i in range(1,higher_quark.GetNbinsX()+1):
                    higher_quark_hist.SetBinContent(i,higher_quark_unc[i-1].nominal_value)
                    higher_quark_hist.SetBinError(i,higher_quark_unc[i-1].std_dev)
                    lower_quark_hist.SetBinContent(i,lower_quark_unc[i-1].nominal_value)
                    lower_quark_hist.SetBinError(i,lower_quark_unc[i-1].std_dev)
                    higher_gluon_hist.SetBinContent(i,higher_gluon_unc[i-1].nominal_value)
                    higher_gluon_hist.SetBinError(i,higher_gluon_unc[i-1].std_dev)
                    lower_gluon_hist.SetBinContent(i,lower_gluon_unc[i-1].nominal_value)
                    lower_gluon_hist.SetBinError(i,lower_gluon_unc[i-1].std_dev)


                factor_quark_hist = higher_quark.Clone()
                factor_gluon_hist = higher_gluon.Clone()
                for i in range(1,higher_quark.GetNbinsX()+1):

                    factor_quark_hist.SetBinContent(i,factor_quark_unc[i-1].nominal_value)
                    factor_quark_hist.SetBinError(i,factor_quark_unc[i-1].std_dev)
                    factor_gluon_hist.SetBinContent(i,factor_gluon_unc[i-1].nominal_value)
                    factor_gluon_hist.SetBinError(i,factor_gluon_unc[i-1].std_dev)

                gStyle.SetOptStat(0)
                c = TCanvas("","",500)
                c.Divide(2,1)

                top = c.cd(1)
                top.SetPad(0.0,0.0,1.0,1.0)
                top.SetFillColor(0)
                top.SetBorderMode(0)
                top.SetBorderSize(2)
                top.SetTickx(1)
                top.SetTicky(1)
                top.SetLeftMargin(0.14)
                top.SetRightMargin(0.055)
                top.SetBottomMargin(0.3)#0.25
                top.SetFrameBorderMode(0)
                #top.SetLogy(1)
                top.cd()

                higher_quark_hist.SetMarkerColor(4)
                higher_quark_hist.SetLineColor(4)
                higher_quark_hist.SetMarkerSize(0.5)
                higher_quark_hist.SetLineStyle(1)

                higher_gluon_hist.SetMarkerColor(2)
                higher_gluon_hist.SetLineColor(2)
                higher_gluon_hist.SetMarkerSize(0.5)
                higher_gluon_hist.SetLineStyle(1)         

                lower_quark_hist.SetMarkerColor(4)
                lower_quark_hist.SetLineColor(4)
                lower_quark_hist.SetMarkerSize(0.5)
                lower_quark_hist.SetLineStyle(2)

                lower_gluon_hist.SetMarkerColor(2)
                lower_gluon_hist.SetLineColor(2)
                lower_gluon_hist.SetMarkerSize(0.5)
                lower_gluon_hist.SetLineStyle(2)           


                bot = c.cd(2)
                bot.SetPad(0.0,0.0,1.0,0.3)
                bot.SetFillColor(0)
                bot.SetBorderMode(0)
                bot.SetBorderSize(2)
                bot.SetTickx(1)
                bot.SetTicky(1)
                bot.SetLeftMargin(0.14)
                bot.SetRightMargin(0.055)
                bot.SetTopMargin(0.045)
                bot.SetBottomMargin(0.4)
                bot.SetFrameBorderMode(0)        

                factor_quark_hist.SetMarkerColor(4)
                factor_quark_hist.SetLineColor(4)
                factor_quark_hist.SetMarkerSize(0.5)
                factor_quark_hist.SetLineStyle(1)

                factor_gluon_hist.SetMarkerColor(2)
                factor_gluon_hist.SetLineColor(2)
                factor_gluon_hist.SetMarkerSize(0.5)
                factor_gluon_hist.SetLineStyle(1)   
                #leg.AddEntry(gluon_data,"Extracted gluon (data)","p")

                factor_quark_hist.SetMinimum(0)
                factor_quark_hist.SetMaximum(2)

                factor_quark_hist.GetYaxis().SetTitle("Forward/Central")        
                factor_quark_hist.GetYaxis().SetRangeUser(0.7,1.3)
                factor_quark_hist.GetXaxis().SetTitleOffset(1)
                factor_quark_hist.GetXaxis().SetTitleSize(0.11)
                factor_quark_hist.GetXaxis().SetLabelSize(0.1)
                factor_quark_hist.GetXaxis().SetLabelOffset(0.03)
                factor_quark_hist.GetYaxis().SetTitleSize(0.1)
                factor_quark_hist.GetYaxis().SetTitleOffset(0.5)
                factor_quark_hist.GetYaxis().SetLabelOffset(0.01)
                top.cd()
                rmax = higher_gluon_hist.GetMaximum()*1.5
                higher_quark_hist.GetYaxis().SetTitle("Normalized to unity")        
                higher_quark_hist.SetMaximum(rmax)
                higher_quark_hist.Draw("HIST")
                higher_gluon_hist.Draw("HIST same")
                lower_quark_hist.Draw("HIST same")
                lower_gluon_hist.Draw("HIST same")

        #         if(var == "ntrk"):
        #             factor_quark_hist.GetXaxis().SetTitle("N_{track}")
        #             leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  

        #         if(var == "bdt"):
        #             factor_quark_hist.GetXaxis().SetTitle("BDT")
        #             leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  

                factor_quark_hist.GetXaxis().SetTitle(var)
                leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  
                leg.AddEntry(higher_quark_hist,"forward quark","p")
                leg.AddEntry(lower_quark_hist,"central quark","p")     
                leg.AddEntry(higher_gluon_hist,"forward gluon","p")
                leg.AddEntry(lower_gluon_hist,"central gluon","p")  
                leg.SetTextFont(42)
                leg.SetFillColor(0)
                leg.SetBorderSize(0)
                leg.SetFillStyle(0)
                leg.SetNColumns(1)
                leg.Draw()        
                myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
                myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
                myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")

                bot.cd()
                factor_quark_hist.Draw("HIST e")
                factor_gluon_hist.Draw("HIST e same")

                if not os.path.exists(mc_type):
                    os.makedirs(mc_type)
                c.Print(mc_type+"/"+var+"_"+str(min)+"_"+mc_type+"_"+str(eta_reweight)+"_distribution.png")
                
                
                c2 = TCanvas("","",500,500)
                
                c2.SetLeftMargin(0.14)
                total_mc.SetMarkerColor(2)
                total_mc.SetLineColor(4)
                total_mc.SetMarkerSize(0.5)
                total_mc.SetLineStyle(1)

                total_data.SetMarkerColor(1)
                total_data.SetLineColor(1)
                total_data.SetMarkerSize(0.5)
                total_data.SetLineStyle(1)  
                
              
                total_mc.GetYaxis().SetTitle("Normalized to unity")        
                total_mc.GetXaxis().SetTitle("#eta") 
                total_mc.GetXaxis().SetTitleOffset(0.8)
                total_mc.GetXaxis().SetTitleSize(0.04)
                total_mc.GetXaxis().SetLabelSize(0.03)
                total_mc.GetXaxis().SetLabelOffset(0.01)
                total_mc.GetYaxis().SetTitleSize(0.04)
                total_mc.GetYaxis().SetTitleOffset(1.4)
                total_mc.GetYaxis().SetLabelOffset(0.03)
                total_mc.GetYaxis().SetLabelSize(0.04)
                total_mc.GetYaxis().SetLabelOffset(0.01) 
                total_mc.SetMaximum(1.5*total_mc.GetMaximum()) 
                
                leg = TLegend(0.6,0.7,0.9,0.9) ##0.6,0.5,0.9,0.7  

                leg.SetTextFont(42)
                leg.SetFillColor(0)
                leg.SetBorderSize(0)
                leg.SetFillStyle(0)
                leg.SetNColumns(1)
                leg.Draw()          
                
                
                leg.AddEntry(total_mc,"MC","l")
                leg.AddEntry(total_data,"Data","l")
                
                total_mc.Draw("HIST e")
                total_data.Draw("HIST e same")
                leg.Draw("same")
                
                myText(0.18,0.84,"#it{#bf{#scale[1.8]{#bf{ATLAS} Internal}}}")
                myText(0.18,0.80,"#bf{#scale[1.5]{#sqrt{s} = 13 TeV}}")
                myText(0.18,0.75,"#bf{#scale[1.5]{pT range: "+str(min)+" - "+str(max)+" GeV}}")  
                
                c2.Print(mc_type+"/"+var+"_"+str(min)+"_"+mc_type+"_"+str(eta_reweight)+"_total.png")

1500
eta 0.4
eta 0.8
500
eta 0.4
eta 0.8
600
eta 0.4
eta 0.8
800
eta 0.4
eta 0.8
1000
eta 0.4
eta 0.8
1200
eta 0.4
eta 0.8


Info in <TCanvas::Print>: png file pythia/eta_500_pythia_0_distribution.png has been created
Info in <TCanvas::Print>: png file pythia/eta_500_pythia_0_total.png has been created
Info in <TCanvas::Print>: png file pythia/eta_600_pythia_0_distribution.png has been created
Info in <TCanvas::Print>: png file pythia/eta_600_pythia_0_total.png has been created
Info in <TCanvas::Print>: png file pythia/eta_800_pythia_0_distribution.png has been created
Info in <TCanvas::Print>: png file pythia/eta_800_pythia_0_total.png has been created
Info in <TCanvas::Print>: png file pythia/eta_1000_pythia_0_distribution.png has been created
Info in <TCanvas::Print>: png file pythia/eta_1000_pythia_0_total.png has been created
Info in <TCanvas::Print>: png file pythia/eta_1200_pythia_0_distribution.png has been created
Info in <TCanvas::Print>: png file pythia/eta_1200_pythia_0_total.png has been created
Info in <TCanvas::Print>: png file pythia/eta_1500_pythia_0_distribution.png has been created
Info in

In [7]:
str(eta_bin[e])

'0.5'